In [ ]:
pip install --upgrade langchain langchain-community openai wikipedia


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=8c572fe4dc9989f5100b99c2e14acc806b769aecc70bbd7e60ee2c8a92d5ef6c
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [ ]:
import requests
from transformers import pipeline
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.tools import WikipediaQueryRun
from langchain.agents import initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from pprint import pprint

In [ ]:
# Define retrieval function
def retrieve_wikipedia_data(query):
    wiki = WikipediaAPIWrapper()
    return wiki.run(query)

In [ ]:
# initialize the LLM and its tokenizer, we are using Flan T5 Large for this
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# function to get the prediction and scores from the LLM, given a prompt
def get_prediction_and_scores(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs =  model.generate(input_ids, output_scores=True, return_dict_in_generate=True, max_length=1000)
    generated_sequence = outputs.sequences[0]

    # get the probability scores for each generated token
    transition_scores = torch.exp(model.compute_transition_scores(
        outputs.sequences, outputs.scores, normalize_logits=True
    )[0])
    return tokenizer.decode(generated_sequence), generated_sequence, transition_scores

In [ ]:
# the input prompt for the LLM containing the question we want to ask
input_text = "Q: Generate an introduction for the Wikipedia page of Kabhi Kabhie (1976 film).\nA:"

In [ ]:
without_flare_output, _, _ = get_prediction_and_scores(input_text)

pprint(without_flare_output)

('<pad>Kabhi Kabhie (1976 film) Kabhi Kabhie is a 1976 Indian Hindi film '
 'directed by A. R. Rahman. The film stars Anupam Kher, Aamir Khan, Aamir '
 'Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir '
 'Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir '
 'Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir '
 'Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir '
 'Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir '
 'Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir '
 'Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir '
 'Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir '
 'Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir '
 'Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir '
 'Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir '
 'Khan, Aamir

In [ ]:
# keep generating tokens until we get a </s> token
while True:
    # get the prediction and scores from the LLM, given the input and all the tokens generated so far
    generated_sequence, tokens, scores = get_prediction_and_scores(input_text)
    # if any token is low in confidence, then do a RAG step
    if torch.min(scores) < 0.1:
        # extract all tokens with high confidence as query
        high_confidence_tokens = tokens[torch.where(scores > 0.1)]
        query = tokenizer.decode(high_confidence_tokens)
        # get the context from the Wikipedia
        context = retrieve_wikipedia_data(query)
        new_input_text = f"Given the below context:\n{context}\n\n Answer the following \n{input_text}\n"
        # get the prediction and scores from the LLM, given the new input
        generated_sequence, _, _ = get_prediction_and_scores(new_input_text)
        input_text = f"{input_text} {generated_sequence}"
        if "</s>" in input_text:
            break

    else:
        # if all tokens are high in confidence, then just add the generated tokens to the input
        input_text = f"{input_text} {generated_sequence}"
        if "</s>" in input_text:
            break
# print the final output
print("Final output:", input_text)

Final output: Q: Generate an introduction for the Wikipedia page of Kabhi Kabhie (1976 film).
A: <pad> Kabhi Kabhie (1976 film) is a 1976 Indian Hindi film directed by A. R. Rahman. The film stars Paresh Rawal, Manoj Bajpayee, and Anupam Kher.</s>


In [ ]:
import textwrap

def print_paragraphs_side_by_side(paragraph1, paragraph2, width=50):
    # Wrap the paragraphs to the specified width
    wrapped1 = textwrap.wrap(paragraph1, width)
    wrapped2 = textwrap.wrap(paragraph2, width)

    # Get the length of the longest wrapped paragraph
    max_lines = max(len(wrapped1), len(wrapped2))

    # Print each line side by side
    for i in range(max_lines):
        part1 = wrapped1[i] if i < len(wrapped1) else ""
        part2 = wrapped2[i] if i < len(wrapped2) else ""
        print(f"{part1:<{width}} | {part2}")

print_paragraphs_side_by_side(input_text, without_flare_output)

Q: Generate an introduction for the Wikipedia page | <pad>Kabhi Kabhie (1976 film) Kabhi Kabhie is a
of Kabhi Kabhie (1976 film). A: <pad> Kabhi Kabhie | 1976 Indian Hindi film directed by A. R. Rahman.
(1976 film) is a 1976 Indian Hindi film directed   | The film stars Anupam Kher, Aamir Khan, Aamir
by A. R. Rahman. The film stars Paresh Rawal,      | Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir
Manoj Bajpayee, and Anupam Kher.</s>               | Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir
                                                   | Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir
                                                   | Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir
                                                   | Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir
                                                   | Khan, Aamir Khan, Aamir Khan, Aamir Khan, Aamir
                                                   | Khan, Aamir Khan, Aamir Khan, Aamir Kha